In [1]:
import torchreid

/home/rickyli/anaconda3/envs/torchreid/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/rickyli/anaconda3/envs/torchreid/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


/home/rickyli/Desktop/dasc7600/deep-person-reid/torchreid/data/datasets/image/market1501.py:38: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  'The current data structure is deprecated. Please '


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [3]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [4]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [5]:
engine.run(
    save_dir="log/resnet50",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training
epoch: [1/60][10/404]	time 0.266 (0.548)	data 0.001 (0.158)	eta 3:41:14	loss 6.9216 (6.7730)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/60][20/404]	time 0.265 (0.408)	data 0.000 (0.079)	eta 2:44:46	loss 6.6372 (6.7989)	acc 3.1250 (0.7812)	lr 0.000300
epoch: [1/60][30/404]	time 0.266 (0.362)	data 0.000 (0.053)	eta 2:25:59	loss 6.5713 (6.7719)	acc 0.0000 (0.5208)	lr 0.000300
epoch: [1/60][40/404]	time 0.263 (0.338)	data 0.000 (0.040)	eta 2:16:29	loss 6.7392 (6.7349)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/60][50/404]	time 0.263 (0.324)	data 0.000 (0.032)	eta 2:10:46	loss 6.7751 (6.7044)	acc 0.0000 (0.3750)	lr 0.000300
epoch: [1/60][60/404]	time 0.263 (0.315)	data 0.000 (0.027)	eta 2:06:58	loss 6.5553 (6.6808)	acc 0.0000 (0.3646)	lr 0.000300
epoch: [1/60][70/404]	time 0.264 (0.308)	data 0.000 (0.023)	eta 2:04:15	loss 6.6552 (6.6752)	acc 0.0000 (0.4018)	lr 0.000300
epoch: [1/60][80/404]	time 0.264 (0.303)	data 0.000 (0.020)	eta 2:02:11	loss 6.5058 (6.6700)	acc 0.0000 (0.

KeyboardInterrupt: 